In [1]:
import pyaerocom as pya
import pandas as pd
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import scipy.stats as stats
import simplejson as json
import numpy as np
import datetime
import copy
import pwlf
import seaborn as sns
from trends_functions import *

In [2]:
pya.browse_database('GEOS-i33p2_HIST*')
#pya.browse_database('GFDL-AM4-amip_HIST*')


Pyaerocom ReadGridded
---------------------
Data ID: GEOS-i33p2_HIST
Data directory: /lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III-2019/GEOS-i33p2_HIST/renamed
Available experiments: ['HIST']
Available years: [1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
Available frequencies ['monthly']
Available variables: ['abs440aer', 'abs550aer', 'abs550bc', 'abs550dust', 'abs550lt1', 'abs550lt2p5', 'abs870aer', 'albsrfc', 'cheaqpso4', 'chegpso4', 'chepmsa', 'clt', 'co', 'co50', 'drybc', 'drydms', 'drydust', 'drymsa', 'drynh3', 'drynh4', 'dryno3', 'dryoa', 'drypm10', 'drypm10dust', 'drypm10ss', 'drypm2p5', 'drypm2p5dust', 'drypm2p5ss', 'dryso2', 'dryso4', 'dryss', 'ec550aer', 'ec550dust', 'emiaoa', 'emibc', 'emico', 'emidms', 'emidust', 'eminh3', 'emioa', 'emipm10', 'emipm10dust', 'emi

['GEOS-i33p2_HIST']

In [19]:
model_id = 'BCC-CUACE_AP3-HIST'
var = 'od550aer'
reader = pya.io.ReadGridded(model_id)
reader


Pyaerocom ReadGridded
---------------------
Data ID: BCC-CUACE_AP3-HIST
Data directory: /lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III-2019/BCC-CUACE_AP3-HIST/renamed
Available experiments: ['HIST']
Available years: [1850, 1860, 1870, 1880, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2014]
Available frequencies ['monthly']
Available variables: ['abs550aer', 'asy550aer', 'clh', 'cll', 'clm', 'clt', 'concbc', 'concdust', 'concoa', 'concso4', 'concss', 'drybc', 'drydust', 'dryoa', 'dryso2', 'dryso4', 'dryss', 'emidust', 'emiss', 'frc0day', 'hur', 'hus', 'laoddust', 'loadbc', 'loaddust', 'loadoa', 'loadso4', 'loadss', 'od550aer', 'od550bc', 'od550dust', 'od550oa', 'od550so4', 'od550ss', 'phis', 'precc', 'precl', 'ps', 'qrs', 'qrs1', 'rlds', 'rlus', 'rlut', 'rlutcs', 'rsds', 'rsdscs', 'rsdt', 'rsdtcs', 'rsns', 'rsns0RF', 'rsns0anthaer', 'rsnscs', 'rsnscs0RF', 'rsnscs0anthaer', 'rsnt', 'rsntcs', 'rsntcs0RF', 'rsntc

In [18]:
mod_data = reader.read_var(var)

VarNotAvailableError: Error: variable absc550aer not available in files and can also not be computed.

In [5]:
mod_data.from_files

['/lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III-2019/MIROC-SPRINTARS_AP3-HIST/renamed/aerocom3_MIROC-SPRINTARS_AP3-HIST_mmrpm2p5_ModelLevel_2010_monthly.nc',
 '/lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III-2019/MIROC-SPRINTARS_AP3-HIST/renamed/aerocom3_MIROC-SPRINTARS_AP3-HIST_rho_ModelLevel_2010_monthly.nc']

In [12]:
#read pickle file
fn = 'cache/'+model_id+'_'+var+'.pkl'
if os.path.isfile(fn):
    print(model_id,'use pickle')
    # for reading also binary mode is important 
    pklfile = open(fn, 'rb')      
    mod_data = pickle.load(pklfile) 
    pklfile.close()

ECMWF_CAMS_REAN use pickle


In [3]:
'''var = 'scon'
source = 'EBASMC'
ymin, ymax = 0, 100
ylabel = 'PM10'
obs_source = source

reader = pya.io.ReadUngridded(source)
obs_data = reader.read(vars_to_retrieve=var)'''

"var = 'scon'\nsource = 'EBASMC'\nymin, ymax = 0, 100\nylabel = 'PM10'\nobs_source = source\n\nreader = pya.io.ReadUngridded(source)\nobs_data = reader.read(vars_to_retrieve=var)"

In [4]:
'''mod_var = 'sconcso4'
source = 'ECMWF_CAMS_REAN'
ymin, ymax = 0, 100
ylabel = 'PM10'
reader = pya.io.ReadGridded(source)
mod_data = reader.read_var(mod_var, ts_type='daily')'''

"mod_var = 'sconcso4'\nsource = 'ECMWF_CAMS_REAN'\nymin, ymax = 0, 100\nylabel = 'PM10'\nreader = pya.io.ReadGridded(source)\nmod_data = reader.read_var(mod_var, ts_type='daily')"

In [5]:
min_dobs = 300 #minimum number of daily observations available in order to keep the station
min_nstat = 2 #minimum number of stations required to compute median
sig = 0.95 #significance
min_dim = 5 #minimum number of days required to compute monthly mean
min_mis = 1 #minimum number of months required to compute seasonal mean
min_siy = 4 #minimum number of seasons required to compute annual mean
nseg = 2 #number of segments if no significant linear trend on the time series is found
use_obs_seg = True #if use same segments for model and bias than the ones found in obs (to be run before)

In [6]:
var =  'sconcso4' #'ang4487aer' 'od550aer' 'od550gt1aer' 'od550lt1aer' 'sconcpm10' 'sconcpm25' 'sconcso4'
if var=='od550aer':
    obs = 'AeronetSunV3Lev2.daily'
    ymin, ymax = 0, 0.8
    ylabel = 'AOD'
    source = 'AeronetSunV3Lev2.daily'
if var=='ang4487aer':
    source = 'AeronetSunV3Lev2.daily'
    ymin, ymax = 0, 2.5
    ylabel = 'AE'
if var=='od550gt1aer':
    source = 'AeronetSDAV3Lev2.daily'
    ymin, ymax = 0, 0.8
    ylabel = 'AOD>1µm'
if var=='od550lt1aer':
    source = 'AeronetSDAV3Lev2.daily'
    ymin, ymax = 0, 0.8
    ylabel = 'AOD<1µm'
if var=='sconcpm10':
    source = 'EBASMC'
    ymin, ymax = 0, 60
    ylabel = 'PM10'
if var=='sconcpm25':
    source = 'EBASMC'
    ymin, ymax = 0, 30
    ylabel = 'PM25'
if var=='sconcso4':
    source = 'EBASMC'
    ymin, ymax = 0, 6
    ylabel = 'SO4'
if var=='wetconcso4':
    source = 'EBASMC'
    ymin, ymax = 0, 50
    ylabel = 'SO4'
    min_dim = 0 
    
obs_source = source
reader = pya.io.ReadUngridded(source)
obs_data = reader.read(vars_to_retrieve=var)

kind = 'obs'
data = obs_data

/home/augustinm/Dev/pyaerocom/pyaerocom/pyaerocom/ungriddeddata.py:123: RuntimeWarning: invalid value encountered in multiply
  self._data = np.empty([num_points, self._COLNO]) * np.nan


ValueError: Invalid input, need instance of UngriddedData, got: <class 'NoneType'>

In [ ]:
#by default, colocate model in space and time
MAP = {}
DF = {}
ALL_TS = {}
regions = pya.region.all()
regions = ['NAMERICA']

#computation region by region
for region in regions:
    f = pya.Filter(region)
    if kind == 'obs':
        try:
            sub = f(data)
            var = data.vars_to_retrieve[0]
        except:
            print('No station found in the area')
            MAP[region] = {
                'name': region,
                'min_lon': pya.Region(region).lon_range[0],
                'max_lon': pya.Region(region).lon_range[1],
                'min_lat': pya.Region(region).lat_range[0],
                'max_lat': pya.Region(region).lat_range[1],
                'trends': {},
                'nmax': 0,
                'stations': [],
            }
            continue
    elif kind == 'mod' or kind == 'bias':
        if obs==None:
            print('kind is model or bias. Needs to pass an obs dataset')
        else:
            try:
                sub = f(obs)
                var = data.var_name
            except:
                print('No station found in the area')
                MAP[region] = {
                    'name': region,
                    'min_lon': pya.Region(region).lon_range[0],
                    'max_lon': pya.Region(region).lon_range[1],
                    'min_lat': pya.Region(region).lat_range[0],
                    'max_lat': pya.Region(region).lat_range[1],
                    'trends': {},
                    'nmax': 0,
                    'stations': [],
                }
                continue

    #for each subset, creates a dataframe containing all stations timeseries
    #first, get station data
    data_all = sub.to_station_data_all()['stats']
    obs_all = copy.copy(data_all)

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    #if model 
    if kind == 'mod':
        if colocate_space: 
            #first, get list of stations name, lat and lon
            stations = { 'name': [], 'lat': [], 'lon': []  }

            for stat in data_all:
                stations['name'].append(stat['station_name'])
                stations['lat'].append(stat['station_coords']['latitude'])
                stations['lon'].append(stat['station_coords']['longitude'])

            data_all = data.to_time_series(longitude=stations['lon'], latitude=stations['lat'],  add_meta=dict(station_name = stations['name']))
        else:
            #creates a filter for the region
            f = pya.Filter(region)
            data_all = f(data)
            med_area = data_all.get_area_weighted_timeseries()
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


    #initialize pandas DataFrame
    df = pd.DataFrame()

    stations = []
    if kind=='mod' and colocate_space==False:
        med = med_area[var]
        top = med
        bottom = med
        ts = med.to_frame()
        ts.set_axis([var+'_'+region], axis=1, inplace=True)
        df = pd.concat([df, ts], axis=1)
    else:
        for i, station in enumerate(data_all):
            stat_name = station.station_name
            clear_output(wait=True)
            print('region: ', region)
            print('station: ', stat_name)
            ts_type = station.ts_type
            #set individual time series as dataframe

            #extract pandas series and convert it to datframe
            ts = data_all[i][var].to_frame()
            ts.set_axis([var+'_'+stat_name], axis=1, inplace=True)

            if ts_type == 'daily':
                if ts.count()[0]>=min_dobs:
                    #concatenates to main dataframe
                    df = pd.concat([df, ts], axis=1)
            else :
                #concatenates to main dataframe
                df = pd.concat([df, ts], axis=1)

            stations.append({
                'name': stat_name,
                'lat': station.latitude,
                'lon': station.longitude
            })
        clear_output(wait=False)

        #caluclates median and envelope with quartiles
        med = df.median(axis=1)
        top = df.quantile(q=0.75,axis=1)
        bottom = df.quantile(q=0.25,axis=1)
        #requires at least n measurements to provide a valid median
        n = df.count(axis=1, numeric_only=False)
        med = med[n>min_nstat]
        top = top[n>min_nstat]
        bottom = bottom[n>min_nstat]
        
    #calculates monthly averages for the plots
    dcount = med.groupby(pd.Grouper(freq='M')).count()
    mmed = med.groupby(pd.Grouper(freq='M')).mean().where(dcount>=min_dim)
    mtop = top.groupby(pd.Grouper(freq='M')).mean().where(dcount>=min_dim)
    mbottom = bottom.groupby(pd.Grouper(freq='M')).mean().where(dcount>=min_dim)

    #seasonal averages
    mcount = mmed.groupby(pd.Grouper(freq='Q')).count()
    smed = mmed.groupby(pd.Grouper(freq='Q')).mean().where(mcount>=min_mis)

    #yearly averages from seasonal averages
    scount = smed.groupby(pd.Grouper(freq='A')).count()
    ymed = smed.groupby(pd.Grouper(freq='A')).mean().where(scount>=min_siy)
    ymed = ymed.shift(-6, freq ='MS')

In [ ]:
#by default, colocate model in space and time
MAP = {}
DF = {}
ALL_TS = {}
regions = pya.region.all()
regions = ['NAMERICA']

#computation region by region
for region in regions:
    f = pya.Filter(region)
    if kind == 'obs':
        try:
            sub = f(data)
            var = data.vars_to_retrieve[0]
        except:
            print('No station found in the area')
            MAP[region] = {
                'name': region,
                'min_lon': pya.Region(region).lon_range[0],
                'max_lon': pya.Region(region).lon_range[1],
                'min_lat': pya.Region(region).lat_range[0],
                'max_lat': pya.Region(region).lat_range[1],
                'trends': {},
                'nmax': 0,
                'stations': [],
            }
            continue
    
    #for each subset, creates a dataframe containing all stations timeseries
    #first, get station data
    data_all = sub.to_station_data_all()['stats']
    obs_all = copy.copy(data_all)

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    #initialize pandas DataFrame
    df = pd.DataFrame()

    stations = []
    for i, station in enumerate(data_all):
        stat_name = station.station_name
        clear_output(wait=True)
        print('region: ', region)
        print('station: ', stat_name)
        ts_type = station.ts_type
        #set individual time series as dataframe

        #extract pandas series and convert it to datframe
        ts = data_all[i][var].to_frame()
        ts.set_axis([var+'_'+stat_name], axis=1, inplace=True)

        if ts_type == 'daily':
            if ts.count()[0]>=min_dobs:
                #concatenates to main dataframe
                df = pd.concat([df, ts], axis=1)
        else :
            #concatenates to main dataframe
            df = pd.concat([df, ts], axis=1)

        stations.append({
            'name': stat_name,
            'lat': station.latitude,
            'lon': station.longitude
        })
    clear_output(wait=False)

    #caluclates median and envelope with quartiles
    med = df.median(axis=1)
    top = df.quantile(q=0.75,axis=1)
    bottom = df.quantile(q=0.25,axis=1)
    #requires at least n measurements to provide a valid median
    n = df.count(axis=1, numeric_only=False)
    med = med[n>min_nstat]
    top = top[n>min_nstat]
    bottom = bottom[n>min_nstat]

In [ ]:
for station in stations:
    print(station['name'])

In [ ]:
df['sconcso4_Algoma'].plot()

In [ ]:
sub = f(data)
var = data.vars_to_retrieve[0]

In [ ]:
data_all = sub.to_station_data_all()['stats']

In [ ]:
data_all[10]

In [21]:
all_mods = ['ECMWF_CAMS_REAN', 'OsloCTM3v1.01-met2010_AP3-HIST', 
            'NorESM2-LM_historical', 'BCC-CUACE_HIST', 'CAM5-ATRAS_AP3-HIST', 
            'GFDL-AM4-amip_HIST', 'CanESM5_historical', 'CESM2_historical',
            'E3SM-1-0_historical', 'IPSL-CM6A-LR_historical']
var = 'od550aer'
all_mods[0]

'ECMWF_CAMS_REAN'

In [37]:
import pickle
import os

for mod_source in all_mods[1:]:
    print(mod_source)
    
    #check if model in cache
    fn = 'cache/'+mod_source+'_'+var+'.pkl'
    if os.path.isfile(fn):
        print('use pickle')
        # for reading also binary mode is important 
        pklfile = open(fn, 'rb')      
        mod_data = pickle.load(pklfile) 
        pklfile.close()
        if 'contactinfo' in mod_data.suppl_info:
            print(mod_data.suppl_info['contactinfo'])
        if 'contact' in mod_data.suppl_info:
            print(mod_data.suppl_info['contact'])
    print()

Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead


OsloCTM3v1.01-met2010_AP3-HIST
use pickle
gunnar.myhre@cicero.oslo.no

NorESM2-LM_historical
use pickle
Please send any requests or bug reports to noresm-ncc@met.no.

BCC-CUACE_HIST
use pickle
Hua Zhang (huazhang@cma.gov.cn) and Zhili Wang (wangzhili1981@126.com)

CAM5-ATRAS_AP3-HIST
use pickle
Hitoshi Matsui (matsui_at_nagoya-u.jp)

GFDL-AM4-amip_HIST
use pickle

CanESM5_historical
use pickle


Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead


ec.cccma.info-info.ccmac.ec@canada.ca

CESM2_historical
use pickle
cesm_cmip6@ucar.edu

E3SM-1-0_historical
use pickle


Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead
Outdated attribute suppl_info. Please use metadata instead


Dave Bader (bader2@llnl.gov)

IPSL-CM6A-LR_historical
use pickle
ipsl-cmip6@listes.ipsl.fr



In [35]:
mod_data.suppl_info

Outdated attribute suppl_info. Please use metadata instead


{'Conventions': 'CF-1.7 CMIP-6.2',
 'creation_date': '2018-07-11T07:36:41Z',
 'tracking_id': 'hdl:21.14100/70325bcd-6e5b-4020-afb7-830828b56bf9',
 'description': 'AOD from ambient aerosols (i.e., includes aerosol water).  Does not include AOD from stratospheric aerosols if these are prescribed but includes other possible background aerosol types. Needs a comment attribute "wavelength: 550 nm"',
 'title': 'IPSL-CM6A-LR model output prepared for CMIP6 / CMIP historical',
 'activity_id': 'CMIP',
 'contact': 'ipsl-cmip6@listes.ipsl.fr',
 'data_specs_version': '01.00.21',
 'dr2xml_version': '1.11',
 'experiment_id': 'historical',
 'experiment': 'all-forcing simulation of the recent past',
 'external_variables': 'areacella',
 'forcing_index': 1,
 'frequency': 'mon',
 'grid': 'LMDZ grid',
 'grid_label': 'gr',
 'nominal_resolution': '250 km',
 'initialization_index': 1,
 'institution_id': 'IPSL',
 'institution': 'Institut Pierre Simon Laplace, Paris 75252, France',
 'license': 'CMIP6 model dat